# Algebraic Geometry in OSCAR: part II

## A look into the OSCAR kitchen: What's cooking?

Together with [Simon Brandhorst](https://www.math.uni-sb.de/ag/brandhorst/) we are currently working on finding an [automorphism of minimal entropy for Enriques surfaces](https://arxiv.org/pdf/1807.09452v2.pdf). To this end, we have implemented and automatized wide parts of the functionality for *elliptic surfaces*.

In [ ]:
using Revise, Oscar

In [ ]:
kk = GF(113) # the finite field (Galois field) with 113 elements
kkt_poly, t_poly = kk[:t] # the polynomial ring in one variable `t` over `kk`
kkt_field = fraction_field(kkt_poly) # the fraction field of that polynomial ring
P, (x, y) = kkt_field[:x, :y]; # the bivariate polynomial ring over `kkt_field`
f = y^2 - x^3 - (1 - t_poly^8)*x # a Weierstrass equation for an elliptic curve
E = elliptic_curve(f, x, y) # the associated elliptic curve as an instance of `EllipticCurve`

We can construct an elliptic surface directly from `E`.

In [ ]:
X = elliptic_surface(E, 2; resolution_strategy=:simultaneous)

Rational points on `E` will give rise to elements in the *Mordell-Weil lattice* of `X`. We have picked some points and make them known on `X`.

In [ ]:
t = kkt_field(t_poly);

basis_mwl = [[62*t^4 + 6*t^3 + 106*t^2 + 75*t + 26, 66*t^6 + 67*t^5 + 37*t^4 + 60*t^3 + 67*t^2 + 101*t + 58], [112*t^4 + 97*t^2 + 98, 104*t^5 + 82*t^3 + 91*t], [t^4 + 112, 87*t^4 + 26], [8*t^4 + 17*t^3 + 11*t^2 + 70*t + 7, 39*t^6 + 17*t^5 + 60*t^4 + 64*t^3 + 41*t^2 + 29*t + 89], [50*t^4 + 104*t^2 + 72, 56*t^6 + 30*t^4 + 36*t^2 + 104], [31*t^4 + 45*t^3 + 60*t^2 + 59*t + 13, 43*t^6 + 18*t^5 + 67*t^4 + 75*t^3 + 10*t^2 + 44*t + 17], [105*t^4 + 43*t^3 + 61*t^2 + 17*t + 7, 20*t^6 + 33*t^5 + 53*t^4 + 9*t^3 + 63*t^2 + 70*t + 89], [62*t^4 + 75*t^3 + 8*t^2 + 107*t + 87, 66*t^6 + 103*t^5 + 103*t^4 + 63*t^3 + 46*t^2 + 37*t + 34]]

basis_mwl = [E(i) for i in basis_mwl]

In [ ]:
Oscar.set_mordell_weil_basis!(X, basis_mwl)

In order to see more of what's going on, we may set up the verbosity.

In [ ]:
set_verbose_level(:EllipticSurface, 5)
set_verbose_level(:MorphismFromRationalFunctions, 5)

For elliptic surfaces we can compute a basis for the **Neron-Severi group** whenever we have found a Mordell-Weil basis.

In [ ]:
# The following one command is only necessary for the correct bookkeeping with further code below; ignore this!
Oscar._trivial_lattice(X;reducible_singular_fibers_in_PP1=Oscar._reducible_fibers_disc(X;sort=false))

# compute the full Neron-Severi lattice of `X`
alg_lat, _, NS = algebraic_lattice(X)
V = ambient_space(NS)

The elements in `alg_lat` are divisors spanning the divisor class group up to *numerical equivalence*. 
The big matrix is the *gram matrix* for their pairwise intersections.

In [ ]:
alg_lat

Since `X` is a smooth, compact surface, the intersection of either two Weil divisors is well defined and we can compute pairwise intersection numbers.

In [ ]:
intersect(alg_lat[1], alg_lat[12])

Even the self-intersection of all generators is known here! But unfortunately this is only known due to some tricks on elliptic surfaces...

In [ ]:
intersect(alg_lat[1], alg_lat[1]) # the fiber of the elliptic fibration has self-intersection 0

In [ ]:
intersect(alg_lat[3], alg_lat[3]) # any rational curve on a K3-surface has self intersection -2

### A dirty speedup for the self-intersection of curves

> **Task 1:** Extend the computation of self-intersection of curves on a smooth surface by means of the [adjunction formula](https://en.wikipedia.org/wiki/Adjunction_formula) and the [Riemann-Roch theorem](https://en.wikipedia.org/wiki/Hirzebruch%E2%80%93Riemann%E2%80%93Roch_theorem) for curves.

Roughly this can be done as follows.

> **Theorem (Riemann-Roch for curves):** Let $C$ be a smooth projective curve and $D$ a divisor on it. Then 
$$\chi(D) = h^0(\mathcal O(D)) - h^1(\mathcal O(D)) = \mathrm{deg}(D) + 1 - g(C)$$
where $g(C) = h^0(\omega_C)$ is the *genus* and $\mathrm{deg}(D) = \int_C c_1(\mathcal O(D))$ is the degree of the divisor.

Applying this to the canonical divisor $D = \omega_C$, we get the 
well known formula $\deg(\omega_C) = 2g(C) - 2$. For the canonical bundle of $C$ we have a formula relating it to the canonical bundle of the ambient surface:

> **Theorem (Adjunction formula):** Let $X$ be a smooth, projective surface and $i : C \hookrightarrow X$ a smooth divisor on it. Then 
$$ \omega_C = i^* (\omega_X \otimes \mathcal O(C)).$$

Intersection theory tells us that $\mathrm{deg}(i^*\mathcal O(C)) = C.C$ is equal to the self-intersection of $C$ in $X$ and that the degree is logarithmic for line bundles on curves. Therefore 
$$2g(C) - 2 = \mathrm{deg}(\omega_C) = \mathrm{deg}(i^* \omega_X) + \deg(i^*\mathcal O(C))) = K_X.C + C.C$$
where $K_X$ is the *canonical divisor* of the surface $X$.

The algorithm would now proceed as follows:
 * Is a canonical divisor of the surface known? If not, throw an error `can not compute self-intersection`.
 * Is the curve smooth and in general position w.r.t $K_X$? If not, throw an error.
 * Compute the genus of $C$.
 * Return $2g(C) - 2 - C.K_X$.
 
For the computation of the genus we may fall back to [implementations in Singular](https://www.singular.uni-kl.de/Manual/4-0-3/sing_1274.htm).

## Weil- and Cartier divisors

> **Definition (algebraic cycle):** Let $X$ be a scheme. An *algebraic cycle* of dimension $k$ is a formal sum 
$$ \sum_{i} a_i \cdot [V_i]$$
where $V_i \subset X$ are $k$-dimensional *subvarieties* of $X$. 

Following **[Ful98]** we will only speak of a variety for reduced and irreducible schemes. The algebraic cycles of dimension $k$ form an Abelian group which will be denoted by $Z_k(X)$. 

> **Definition (Weil divisor):** Let $X$ be an equidimensional scheme of dimension $d$. A *Weil divisor* is an algebraic cycle on $X$ of dimension $d-1$.

Some divisors are more special than others:

> **Definition (Cartier divisor):** An *effective Cartier divisor* is given by an ideal sheaf $\mathcal I$ on $X$ which is locally principal for some non-zero divisor. A *Cartier divisor* is a formal linear combination of effective Cartier divisors.

In other words: For every point $p \in X$ there exists an open neighborhood $U \supset p$ and a non-zero divisor $f \in \mathcal O(U)$ such that $\mathcal I(U) = \langle f \rangle \subset \mathcal O(U)$.

For any two such ''local trivializations'' $(U, f)$ and $(V, g)$ of an effective Cartier divisor $C$ there is a unit $u \in \mathcal O(U \cap V)$ such that $f|_{U \cap V} = u \cdot g|_{U \cap V}$. These give rise to transition functions of a line bundle on $X$ which we will denote by $\mathcal O(C)$. This extends naturally to all Cartier divisors by taking common refinements.

We also have a Cartier divisor floating around, coming as exceptional divisor of the resolution of singularities; but we first have to dig it out of the internals of the elliptic surface.

In [ ]:
Exc_amb = X.ambient_exceptionals[1] # a Cartier divisor on some smooth ambient space of `X`
Exc = pullback(X.inc_Y, Exc_amb) # the pullback of that divisor to `X` itself
first.(Exc.(affine_charts(X)))

Any Cartier divisor gives rise to a Weil divisor:

In [ ]:
Exc_weil = weil_divisor(Exc)
irreducible_decomposition(Exc_weil)

The converse does not hold in general. However, it holds on *smooth* schemes and for every Weil divisor $D$ one may ask: ''Is $D$ actually Cartier?''

> **Task 2:** Implement an algorithm which, given a Weil divisor $D$, decides whether $D$ is Cartier and, if so, computes a local trivialization for $D$.

The point in this is, as we shall see later, that while the intersection of two Weil divisors is usually not well defined, the intersection of a Cartier- and a Weil divisor always is. In fact, Cartier divisors play a crucial role in the development of intersection theory.

### An automorphism of the K3 surface

In our project with Simon Brandhorst we are constructing a very specific automorphism of a K3 surface (which is a lift to the universal cover of the Enriques surface automorphism proposed by Oguiso and Yu). K3 surfaces enjoy various favourable properties which allow us (more or less) to identify an automorphism $f \colon X \to X$ from its ''fingerprint'' given by the action on the Neron-Severi lattice

$$ f_* \colon \mathrm{NS}(X) \to \mathrm{NS}(X).$$

This should be thought of as something like a *hash function*: Any two given automorphisms $f$ and $g$ are very likely to agree if their actions on the Neron-Severi lattice do. However, given $f_*$ as above, it is almost impossible to reconstruct $X$ and $f$. This is basically what the whole paper with Simon is about!

Let us have a look at the reconstruction of $f$.

In [ ]:
# the matrix representing the Oguiso-Yu automorphism w.r.t the basis of V
fOYonAlgX = QQ[5 1 0 -2 1 -1 0 -1 1 0 -1 0 -1 1 0 0 0 2; 2 -1 1 -2 2 0 1 -2 2 0 -2 0 -2 3 0 1 1 3; 4 1 0 -1 1 -1 0 -1 1 0 -1 0 -1 1 0 0 0 2; 5 1 0 -2 1 -1 0 -2 1 0 -1 0 -1 1 0 0 0 2; 1 0 -1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0; 5 1 0 -2 1 -1 0 -1 1 -1 -1 0 -1 1 0 0 0 2; 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0; 4 1 0 -2 1 0 0 -1 1 0 -1 0 -1 1 0 0 0 2; 1 0 0 0 0 0 -1 0 0 0 0 0 0 0 0 0 0 0; 10 -1 2 -7 5 -2 2 -4 5 1 -5 0 -5 7 0 2 2 9; 10 3 -1//2 -7//2 1//2 -3//2 -1//2 -5//2 1//2 -1//2 -2 0 -2 2 0 0 0 2; 7 1 1 -3 1 -1 0 -2 2 0 -3 0 -2 2 0 1 1 3; 4 0 1 -3 2 0 1 -3 1 1 -2 0 -3 3 1 1 0 3; 8 2 1 -3 1 -1 0 -2 2 0 -3 0 -2 2 -1 1 1 3; 10 0 2 -6 4 -2 1 -3 4 0 -5 0 -3 5 -1 2 2 7; 9 1 1//2 -9//2 5//2 -3//2 3//2 -7//2 5//2 1//2 -3 -1 -4 5 0 1 1 5; -3 -2 1 0 1 0 1 0 1 0 0 0 0 1 0 0 1 1]

So that is the fingerprint we need to realize. We have constructed a candidate for the automorphism; let's load it.

In [ ]:
coord_imgs = load("coord_imgs.mrdi")
coord_imgs = [evaluate(fr, gens(ambient_coordinate_ring(weierstrass_chart_on_minimal_model(X)))) for fr in coord_imgs]

In [ ]:
aut = morphism_from_rational_functions(X, X,
            weierstrass_chart_on_minimal_model(X),
            weierstrass_chart_on_minimal_model(X),
            coord_imgs; check=false
           );

Weil divisors are covariant in nature with respect to proper maps. 

> **Definition (proper pushforward):** Let $f \colon X \to Y$ be a proper morphism of algebraic schemes, $P \in X$ a point of dimension $p$ and $Q = f(P) \in Y$ its image. The pushforward of the cycle $[P] \in Z_p(X)$ is defined as 
$$ f_*([P]) = \begin{cases} [R(P):R(Q)] \cdot [Q] &\mathrm{ if } \quad \dim Q = \dim P \\ 0  & \mathrm{ otherwise.} \end{cases}$$
The general pushforward for algebraic cycles is defined by linear extension to $Z_p(X)$.

Here we denote by $R(P)$ the field of rational functions on the subvariety of $X$ given by $P$ and vice versa for $Q$. 

For an *isomorphism* we can skip the computation of the degree of field extensions and just compute the image of a point.

In [ ]:
set_attribute!(aut, :is_isomorphism=>true)
D = alg_lat[3]
pf_D = pushforward(aut, D)

> **Task 3:** Extend the functionality for pushforward of algebraic cycles to more than just isomorphisms.

One possibility would, for instance, be blowdown maps where we know that they are proper and birational. More generally, we do not have a good method to compute the degree of the field extension directly. But one could certainly think of one or two dirty workarounds for this...

For K3 surfaces we can do more than just push forward a divisor: We can represent the image divisor in our chosen basis for the Neron-Severi lattice!

In [ ]:
rep_pf_D = basis_representation(X, pf_D)

For elliptically fibered K3 surfaces we have an automated and optimized method to do everything at once.

In [ ]:
pf_aut = Oscar.pushforward_on_algebraic_lattices(aut) # CAUTION: This command takes some time for computation!

That finding a basis representation for a divisor actually works is due to a rather remarkable property of K3 surfaces: The notions of rational-, algebraic-, and numerical equivalence agree. Consequently, we can compute a basis representation for any divisor from its intersection numbers with the generators of the Neron-Severi lattice.

Note, however, that for a general surface it is almost impossible to determine whether or not two given divisors are even rationally equivalent!

Before we move on, let us once compare the induced map on $\mathrm{NS}(X)$ for the given automorphism `aut` with the fingerprint of `f`:

In [ ]:
matrix(pf_aut)

In [ ]:
fOYonAlgX

As you can see, we did not get completely lucky this time. Some more adjustments seem to be needed.

### A brief review of rational equivalence

Let $X$ be a scheme, $U \subset X$ an affine open subset, and $f \in \mathcal O(U)$ a function. 
For a prime ideal $P \subset \mathcal O(U)$ of codimension one we define the *order* of $f$ at 
the point $P$ as 

$$ \mathrm{ord}_P(f) = \mathrm{length}(\mathcal O_{X, P}/\langle f\rangle)$$

where $\mathcal O_{X, P} = \mathcal O(U)_P$ is the localization at the complement of $P$.
This ring is local of dimension one, so that the quotient by $\langle h\rangle$ is zero-dimensional and hence has finite length.


If $X$ is a variety, then this extends naturally to rational functions on $X$ and we get the following notion.

> **Definition (principal divisor):** Let $W$ be a variety and $f \in R(W)$ a rational function. The *principal divisor* associated to $f$ is 
$$\mathrm{div}(f) = \sum_{P \subset W} \mathrm{ord}_P(f) \cdot [P].$$

Moreover, two algebraic cycles can be related by the following equivalence.

> **Definition (rational equivalence):** Two algebraic cycles $D, E \in Z_k(X)$ on a scheme $X$ are called *rationally equivalent* $D \sim E$ if there exist a finite number of subvarieties $W_i \subset X$ of dimension $k+1$ and rational functions $f_i \in R(W_i)$ such that 
$$ D - E = \sum_i \mathrm{div}(f_i).$$

The quotient $Z_k(X)/\sim$ will be denoted by $A_k(X)$, the $k$-th *Chow group* of $X$.

> **Proposition (pushforward on Chow groups):** The pushforward for proper maps descends to a morphism of Chow groups.

See e.g. **[Ful98]**, Theorem 1.4.

We do already have a data type for rational functions on varieties:

In [ ]:
RX = function_field(X)

Elements of this field are represented on the different charts `U` of `X` as fractions of polynomials for the `ambient_coordinate_ring` of `U`. This is mostly due to the fact that we have no implementation of fraction fields for quotients of polynomial rings at this time. 

In [ ]:
W = weierstrass_chart_on_minimal_model(X)
R = ambient_coordinate_ring(W)
(x, y, t) = gens(R)
f = RX(x) # x as a rational function on `X`; potentially with poles outside `W`

Let us look at a representative of $f$ in another chart:

In [ ]:
simp_cov = Oscar.simplified_covering(X)
U = simp_cov[5]
f[U] # Note that we are using rectangular brackets for this at the moment!

Note that representatives are not necessarily automatically reduced.

In [ ]:
g = RX(first(gens(modulus(OO(W)))))
g[U]

In [ ]:
is_zero(g)

Determining the principal divisor associated to a rational function is something which is actually still missing from our implementations!

> **Task 4:** Implement an algorithm which computes $\mathrm{div}(f)$ for a rational function $f \in R(X)$ on a variety $X$.

However, we already have a method to determine the order of vanishing of a rational function on a divisor which might be helpful.

In [ ]:
I = ideal(OO(W), y)
decomp = primary_decomposition(I)
P = decomp[1][1]

In [ ]:
D = weil_divisor(ideal_sheaf(X, W, P))
order_of_vanishing(f, D)

In [ ]:
order_of_vanishing(f^(-2), D)

## More on Cartier divisors

While Weil divisors are of homological nature (i.e. they have a covariant behaviour), Cartier divisors are cohomological: Given a morphism of schemes $f \colon X \to Y$ and a Cartier divisor $C$ on $Y$ there should be a natural notion of ''pulling back'' $C$. Unfortunately, this is not quite the case.

### Cartier divisors on varieties
On a *variety* $X$ we can define Cartier divisors to be given by a collection $(U_i, f_i)$ where $f_i \in R(X)$ is a rational function and for any pairs $U_i \cap U_j$ the quotient $f_i/f_j \in \mathcal O(U_i \cap U_j)^*$.
In theory, this spares us the detour to complete the monoid of effective Cartier divisors.

Recall that to each Cartier divisor $C$ we can associate a line bundle $\mathcal L = \mathcal O(C)$. 
A rational function $f \in R(X)$ determines a *principal* Cartier divisor and we may call two Cartier divisors 
*rationally equivalent*, if their difference is principal. 

> **Lemma:** On a variety $X$ there is a $1:1$-correspondence between Cartier divisors up to rational equivalence and isomorphism classes of line bundles. 

In order to get back a Cartier divisor from a line bundle $\mathcal L$ we may construct a *rational section* $s$ of $\mathcal L$: Start on one chart $U_0$ of a local trivialization of $\mathcal L$ and extend the local algebraic section $e$ for this trivialization as a rational section to all of $X$. In the notation above, the associated Cartier divisor will then be given by the collection $(U_j, f_j/f_0)$. We denote it by $c_1(\mathcal L)$. 

### Intersection of a Cartier divisor with an algebraic cycle

Let $X$ be a scheme and $D = \sum_i a_i \cdot W_i \in Z_k(X)$ an algebraic cycle with $W_i$ irreducible. For a Cartier divisor $C = \{(U_j, f_j)\}_{j=1}^N$ we may define the intersection $C \cap D$ as follows.

 * We reduce to the case of irreducible cycles by $C \cap (\sum_i a_i \cdot W_i) = \sum_i a_i \cdot (C \cap W_i)$.
 * For an irreducible cycle $\iota \colon W \hookrightarrow X$ we set 
   $$C \cap W := \iota_*(c_1(\iota^* \mathcal O(C))\cap [W]).$$
 * $\iota^*\mathcal O(C)$ is the pullback of the line bundle $\mathcal O(C)$ associated to $C$. 
 * $\iota_*$ denotes the proper pushforward along the closed embedding $\iota$.
 * $[W]$ denotes the fundamental class of $[W]$. 
 * $c_1(\mathcal L) \cap [W]$ is the Weil divisor on $W$ associated to the Cartier divisor $c_1(\mathcal L)$.

### More general intersections of divisors on surfaces

The current implementation has a method `intersect(D::AbsAlgebraicCycle, C::CartierDivisor)`, but it is not fully implemented yet! Let's have a look at some examples.

In [ ]:
Z = intersect(alg_lat[1], Exc)
integral(Z)

In [ ]:
Z = intersect(alg_lat[2], Exc)
integral(Z)

In [ ]:
Z = intersect(alg_lat[3], Exc)

In [ ]:
Z = intersect(alg_lat[12], Exc)
integral(Z)

In [ ]:
Z_decomp = irreducible_decomposition(Z)

In [ ]:
integral(ans)

This leads us to some more tricky tasks to achieve this week.

> **Task 5:** Fill the gaps in the implementation of `intersect` for an `EffectiveCartierDivisor` and an `AbsAlgebraicCycle`.

**Warning:** This is a deep dive! We will need some preparation for that.

> **Task 6:** Speed up the implementation for `irreducible_decomposition`. 

It seems that a lot of time is spent in the generic method. But for many useful applications it should be possible to use more efficient shortcuts!

> **Task 7:** Implement a method to construct the line bundle $\mathcal O(C)$ associated to a Cartier divisor.

This is rather technical. We do have support for coherent sheaves, though, but maybe for the purpose of intersection theory, we can avoid actually carrying out this nasty step?

> **Task 8:** Implement the reconstruction of a Cartier divisor for a given line bundle $\mathcal L$ on a variety $X$.